# Colorizing Australia

In this assignment, you will implement the backtracking search for colorizing the Australia on your own.

In [2]:
class CSP(object):
    def __init__(self):
        self.variables = []
        self.domains = {}
        self.factors = {}

    def add_variable(self, variable, domain):
        """
        Takes variable and its domain and adds them to the CSP
        :param variable: any type
        :param domain: list of possible values for teh variable
        """
        if variable in self.variables: raise ValueError('This CSP already contains variable', variable, ', please use another name or check the consistency of your code.' )
        if type(domain) is not list: raise ValueError('Domain should be a list.' )
        self.variables.append(variable)
        self.domains[variable] = domain

    def add_factor(self, variables, factor_function):
        """
        Takes variables and the potential for these variables and adds them to the CSP
        :param variables: set of variables
        :param potential: potential function taking values of the variables as input and returning 
        non-negative value of the potential  
        """

        if type(variables) not in (list, set): raise ValueError('Variables should be a list or a set.' )
        if type(variables) is list: variables = frozenset(variables)

        if variables in self.factors.keys():
            self.factors[variables].append(factor_function)
        else:
            self.factors[variables] = [factor_function]

Here is the code constructing CSP for you. Examine it carefuly, make sure you understand all the srtuctures used.

In [3]:
csp_Australia = CSP()
provinces = ['WA', 'NT', 'Q', 'NSW', 'V', 'SA', 'T']
neighbors = {
    'SA' : ['WA', 'NT', 'Q', 'NSW', 'V'],
    'NT' : ['WA', 'Q'],
    'NSW' : ['Q', 'V']
}

colors = ['red', 'blue', 'green']

def are_neighbors(a, b):
    return (a in neighbors and b in neighbors[a]) or (b in neighbors and a in neighbors[b])

for p in provinces:
    csp_Australia.add_variable(p, colors)
for p1 in provinces:
    for p2 in provinces:
        if are_neighbors(p1, p2):
            # Neighbors cannot have the same color
            csp_Australia.add_factor([p1, p2], lambda x, y : x != y)

Here you will implement backtracking search with advanced forward checking and without propagation through a singleton domain for a good reason.

Backtracking search will work like the following: it takes partial assignment is a form of all possible values to the variables (initially - full domains of the CSP). Next, we will do assignments by eliminating everything except the assigned value from the variable's domain. This way, we already enforce Prop-1 by design. 

Here is scp_solver class where you should add your code.

In [98]:
import copy

class csp_solver():
    def solve(self, csp):
        self.csp = csp

        # Dictionary for storing all valid assignments. Note, unlike standard CSP when we
        # are interested only in one valid assignment, for this problem you need to
        # find all valid assignments
        self.valid_assignments = []

        # Number of all valid assignments

        # Number of times backtracking operation is called
        self.num_operations = 0
        
        for province in csp.domains:
            for color in colors:
                temp_domains = copy.deepcopy(csp.domains)
                temp_domains[province] = [color]
                self.backtrack(province, temp_domains)
                print("next combination")

        if self.valid_assignments != []:
            print('Found %d optimal assignments in %d operations' % (len(self.valid_assignments), self.num_operations))
            for assignment in self.valid_assignments:
                print(assignment)
        else:
            print('No assignments was found.')


    def backtrack(self, next_variable, partial_assignment):
        """
        
        2c.
        
        Here you will implement backtracking search with arc consistency (advanced forward checking) and without propagation 
        through a singleton domain for a good reason.
        
        Backtracking search will work like the following: it takes partial assignment is a form of 
        all possible values to the variables (initially - full domains of the CSP). Next, we will do assignments
        by eliminating everything except the assigned value from the variable's domain. This way, we already enforce
        Prop-1 by design. 
        
        If the valid assignment is found, add it to self.valid_assignments in a form of (assignment:}
        
        :param next_variable: recently assigned variable or the variable for which the domain has been reduced
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of possible values}
        """
        self.num_operations += 1

        #-------- YOUR CODE HERE ----------
        partial_assignment = self.forward_checking(next_variable, partial_assignment, self.csp.factors)
#         print("after forward_checking")
        if partial_assignment == {}: # empty domains, continue
            return 
        
        next_variable = self.choose_next_variable(partial_assignment)
        
        if len(partial_assignment[next_variable]) == 1: # complete assignment, largest domain equals one
            self.valid_assignments.append(partial_assignment) # update best
#             print( self.valid_assignments)
            return
        
        
        for color in partial_assignment[next_variable]:
            partial_assignment[next_variable] = [color]
            self.backtrack(next_variable, partial_assignment)
                    

    def choose_next_variable(self, partial_assignment):
        """
        
        2a.
            
        As Prop-1 is already implemented, we will use different heuristics and return the variable with the _largest_ domain.
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of values}
        :return: variable for partial assignment
        """
        # -------- YOUR CODE HERE ----------
        var = max(partial_assignment, key=len)
        return var 

    def forward_checking(self, assigned_variable, partial_assignment, factors):
        """
        
        2b.
        
        Implements forward checking on steroids. Checks if any domain contains values inconsistent with current assignment
        and eliminate these variables from the domain. As a result of this domain reduction there could be another
        inconsistency in the domains. Eliminate them recursively by keeping track of the reduced domain and calling forward_checking
        as a recursion. 
        
        This wild version of forward checking is called arc consistency and is one of the most efficient implementation of the
        forward checking idea for CSP.
       
        :param assigned_variable: recently assigned variable or the variable for which the domain has been reduced
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of values}
        :param factors: a dictionary containing factoors of the CSP if the form of {frozenset(variables):list of constraint functions}
        :return: a dictionary of partial assignments with reduced domains
        """
        # -------- YOUR CODE HERE ----------
        
        new_partial_assignment = copy.deepcopy(partial_assignment)
        pass_current_province = False
        for province in partial_assignment:
            print(len(partial_assignment))
            if province == assigned_variable:
                pass_current_province = True
                continue
            if pass_current_province:
                if frozenset([assigned_variable, province]) in factors:
#                     print(assigned_variable, province)
#                     print("yes")
                    new_domain = list(set(partial_assignment[province]) - set(partial_assignment[assigned_variable]))
                    if new_domain == []:
                        print("return {}")
                        return {}
                    new_partial_assignment[province] = new_domain
                    new_partial_assignment = self.forward_checking(province, new_partial_assignment, factors)
                    if new_partial_assignment == {}:
                        return {}
    
                    
        return new_partial_assignment

In [99]:
solver = csp_solver()
solver.solve(csp_Australia)

7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
return {}
7
7
7
7
7
7
7
7
7
7
7
7
return {}
next combination
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


RecursionError: maximum recursion depth exceeded in comparison